# **Install and import packages**

In [ ]:
!pip install bert-extractive-summarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
!pip install spacy

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [ ]:
!pip install fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.8 MB/s eta 0:00:00


# **Import packages**

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import openpyxl
import xlrd
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import fitz

In [ ]:
import re

In [ ]:
import setuptools

In [ ]:
import torch
import gc

In [ ]:
from transformers import pipeline

In [ ]:
from summarizer import Summarizer, TransformerSummarizer

In [ ]:
# Code to read csv file into Colaboratory:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


# **Código**

In [ ]:
pd.options.display.max_colwidth = 10000

In [ ]:
a = os.listdir('/drive/MyDrive/Doutorado_v2/Staging/')
df_files = pd.DataFrame(list(a), columns=['name_file'])
df_files[['a', 'b', 'c']] = df_files['name_file'].str.split('_', 3, expand=True)
df_files[['N', 'e']] = df_files['c'].str.split('.', 2, expand=True)
df_files['files_in'] = df_files['a'] + '_' + df_files['b'] + '_' + df_files['N']
df_files = df_files.drop(['a', 'b', 'c', 'e'], axis=1)
df_files['N'] = df_files['N'].astype(int)
df_files = df_files.sort_values(by=['N'], ascending=[True]).reset_index()
df_files = df_files.drop(['index'], axis=1)

z = (df_files['N'].min()) + 9
df_files = df_files[(df_files['N'] <= z)]

df_files.head(10)

,name_file,N,files_in
0,file_reportagens_26.xlsx,26,file_reportagens_26
1,file_reportagens_27.xlsx,27,file_reportagens_27
2,file_reportagens_28.xlsx,28,file_reportagens_28
3,file_reportagens_29.xlsx,29,file_reportagens_29
4,file_reportagens_30.xlsx,30,file_reportagens_30
5,file_reportagens_31.xlsx,31,file_reportagens_31
6,file_reportagens_32.xlsx,32,file_reportagens_32
7,file_reportagens_33.xlsx,33,file_reportagens_33
8,file_reportagens_34.xlsx,34,file_reportagens_34
9,file_reportagens_35.xlsx,35,file_reportagens_35


In [ ]:
path_in_excel = '/drive/MyDrive/Doutorado_v2/Staging/'
path_out_excel = '/drive/MyDrive/Doutorado_v2/Resumos_BERT'

In [ ]:
%%time
bert_model = Summarizer()

CPU times: user 6.39 s, sys: 4.08 s, total: 10.5 s
Wall time: 23.5 s


In [ ]:
def chr_remove(old, to_remove):
    new_string = old
    for x in to_remove:
        new_string = new_string.replace(x, '')
    return new_string

In [ ]:
%%time
#for i in range(1, 2):
for i in range(0, len(df_files)):
#i = 0
    name_file_in = df_files.iloc[i]['name_file']
    file_excel_in = os.path.join(path_in_excel, name_file_in)
    reportagens_index = pd.read_excel(file_excel_in, sheet_name = "Sheet1")

    for k in range(0, len(reportagens_index)):
        name_file_df = reportagens_index.iloc[k]['name_file']
        path_in = '/drive/MyDrive/Doutorado_v2/Files_in_PDF'
        file_in = os.path.join(path_in, name_file_df)

        text = ''
        with fitz.open(file_in) as doc:
            for page in doc:
                text+= page.get_text()

        page004 = text.split("\n")[4:]
        page004 = repr(page004)
        page004 = chr_remove(page004, "',")
        page004 = chr_remove(page004, '"')
        page004 = " ".join(page004.split())

        reportagens_index.loc[k, 'text_full'] = page004

    df = reportagens_index[['text_full']]

    # Creation the list with new long block
    max_length = 50  # minimum characters in each block
    j = 0
    bodies = []
    while j < len(df):
        body = ""
        body_empty = True
        while (len(body) < max_length) and (j < len(df)):
            if body_empty:
                body = df.loc[j,'text_full']
                body_empty = False
            else: body += " " + df.loc[j,'text_full']
            j += 1
        bodies.append(body)

    bert_summary = []
    for p in range(len(bodies)):
        bert_summary.append(''.join(bert_model(bodies[p], min_length=50)))
        reportagens_index.loc[p, 'Sumarization'] = bert_summary[p]
        reportagens_index.loc[p, 'text_full'] = 'b'

    name_file_out = df_files.iloc[i]['name_file']
    file_excel_out = os.path.join(path_out_excel, name_file_out)
    reportagens_index.to_excel(file_excel_out, index=False)

    torch.cuda.empty_cache()
    del body, bert_summary, bodies, df
    gc.collect()

    os.remove(file_excel_in)
    print(i)

0
1
2
3
4
5
6
7
8
9
CPU times: user 3h 27min 32s, sys: 1min 2s, total: 3h 28min 34s
Wall time: 3h 32min 24s


In [ ]:
reportagens_index.head(10)